In [ ]:
#This checks the python version and installs all necessary dependencies that weren't already preinstalled in Deepnote
!python --version
#!pip install tensorflow
!pip install opencv-python
#!pip install tensorflow

Python 3.7.3
     |████████████████████████████████| 28.2 MB 3.4 MB/s eta 0:00:01
You should consider upgrading via the '/opt/venv/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import os
import sklearn
import sklearn.model_selection
import matplotlib.pyplot as plt

In [0]:
training_images = []
training_labels = []

label_dict = {0:"Electrocautery Tool", 1:"Shaver Tool", 2:"No Tool"}

for i in os.listdir("training_images/electrocautery_training/electrocauteryMin3_electrocauterysegs/"):
    if "png" in str(i):
        print("reading image: " + "training_images/electrocautery_training/electrocauteryMin3_electrocauterysegs/" + str(i))
        img = cv2.imread("training_images/electrocautery_training/electrocauteryMin3_electrocauterysegs/" + str(i))
        img = cv2.resize(img, (256,256))
        training_images.append(img)
        training_labels.append(0)

for i in os.listdir("training_images/shaver_training/electrocauteryMin1_shaverimages/"):
    if "png" in str(i):
        print("reading image: " + "training_images/shaver_training/electrocauteryMin1_shaverimages/" + str(i))
        img = cv2.imread("training_images/shaver_training/electrocauteryMin1_shaverimages/" + str(i))
        img = cv2.resize(img, (256,256))
        training_images.append(img)
        training_labels.append(1)
        
for i in os.listdir("training_images/shaver_training/electrocauteryMin3_shaverimages/"):
    if "png" in str(i):
        print("reading image: " + "training_images/shaver_training/electrocauteryMin3_shaverimages/" + str(i))
        img = cv2.imread("training_images/shaver_training/electrocauteryMin3_shaverimages/" + str(i))
        img = cv2.resize(img, (256,256))
        training_images.append(img)
        training_labels.append(1)
        
for i in os.listdir("tissue/"):
    if "png" in str(i):
        print("reading image: " + "tissue/" + str(i))
        img = cv2.imread("tissue/" + str(i))
        img = cv2.resize(img, (256,256))
        training_images.append(img)
        training_labels.append(2)


reading image: training_images/shaver_training/electrocauteryMin3_shaverimages/frame10724.png
reading image: training_images/shaver_training/electrocauteryMin3_shaverimages/frame6619.png
reading image: training_images/shaver_training/electrocauteryMin3_shaverimages/frame12790.png
reading image: training_images/shaver_training/electrocauteryMin3_shaverimages/frame12437.png
reading image: training_images/shaver_training/electrocauteryMin3_shaverimages/frame8976.png
reading image: training_images/shaver_training/electrocauteryMin3_shaverimages/frame7635.png
reading image: training_images/shaver_training/electrocauteryMin3_shaverimages/frame6270.png
reading image: training_images/shaver_training/electrocauteryMin3_shaverimages/frame8491.png
reading image: training_images/shaver_training/electrocauteryMin3_shaverimages/frame12763.png
reading image: training_images/shaver_training/electrocauteryMin3_shaverimages/frame9638.png
reading image: training_images/shaver_training/electrocauteryMin3_

reading image: tissue/frame241.png
reading image: tissue/frame2775.png
reading image: tissue/frame1152.png
reading image: tissue/frame3868.png
reading image: tissue/frame1867.png
reading image: tissue/frame2097.png
reading image: tissue/frame3816.png
reading image: tissue/frame2554.png
reading image: tissue/frame922.png
reading image: tissue/frame828.png
reading image: tissue/frame3178.png
reading image: tissue/frame3796.png
reading image: tissue/frame3160.png
reading image: tissue/frame2664.png
reading image: tissue/frame3419.png
reading image: tissue/frame3063.png
reading image: tissue/frame4284.png
reading image: tissue/frame3056.png
reading image: tissue/frame858.png
reading image: tissue/frame2082.png
reading image: tissue/frame1371.png
reading image: tissue/frame961.png
reading image: tissue/frame321.png
reading image: tissue/frame69.png
reading image: tissue/frame3141.png
reading image: tissue/frame3104.png
reading image: tissue/frame3913.png
reading image: tissue/frame3136.png


In [ ]:
training_images = np.array(training_images)
training_labels = np.array(training_labels)

In [ ]:
def create_model():
    model = tf.keras.Sequential()
    #First block
    model.add(tf.keras.layers.Conv2D(32, (3,3), input_shape=(256,256, 3), activation='relu', padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.0001)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.0001)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.4))
    
    #Second block
    model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.0001)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.0001)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.4))
    
    #Third block
    model.add(tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.0001)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.0001)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.4))
    
    
    #Fully Connected Layers
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(3, activation='softmax'))
    
    #auc = tf.keras.metrics.AUC()
    
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

temp_model = create_model()
print(temp_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
dropout (Dropout)            (None, 64, 64, 32)        0

In [ ]:
print(training_images.shape)
print(training_labels.shape)

(21286, 256, 256, 3)
(21286,)


In [ ]:

k_fold_acc_data = []
k_fold_histories = []
k_fold_classification_reports = []

k = 10
#Set the seed to a specifc value for reproducibility
random_seed = 22
np.random.seed(random_seed)

kfold = sklearn.model_selection.StratifiedKFold(n_splits=k, shuffle=True, random_state=random_seed)
fold_num = 0
for train, test in kfold.split(training_images, training_labels):
    model = create_model()
    
    callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=5)
    history = model.fit(training_images[train], training_labels[train], epochs=200, callbacks=[callback])
    
    k_fold_histories.append(history)
    
    scores = model.evaluate(training_images[test], training_labels[test])
    k_fold_acc_data.append(scores[1] * 100)

    predictions = []
    for i in training_images[test]:
        predictions.append(np.argmax(model.predict(np.expand_dims(i,0))))
    k_fold_classification_reports.append(sklearn.metrics.classification_report(training_labels[test], predictions, target_names=["Electrocautery Tool", "Shaver Tool", "No Tool"]))
    
    fold_num += 1


KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
#Print the accuracy results and average accuracy with +/-
acc_file = open("Model_Data/Fold_Test_Accuracy.txt", 'w+')
for i, enum in enumerate(k_fold_acc_data):
    print("Fold " + str(i) + " Accuracy: " + str(enum) + "%")
    acc_file.write("Fold " + str(i) + " Accuracy: " + str(enum) + "%" +"\n")
print("Average Accuracy: " + str(np.mean(k_fold_acc_data))  + " (+/- " + str(np.std(k_fold_acc_data)) + ")%")
acc_file.write("Average Accuracy: " + str(np.mean(k_fold_acc_data))  + " (+/- " + str(np.std(k_fold_acc_data)) + ")%" +"\n")
acc_file.close()

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
#Plot and show the history for each fold
for i, enum in enumerate(k_fold_histories):
    plt.plot(enum.history['accuracy'])
    plt.title("Fold " + str(i) + " Accuracy")
    plt.ylabel("Accuracy")
    plt.xlabel("Epoch")
    plt.show()
    plt.savefig("Model_Data/Fold_" + str(i) + "_Accuracy.png")

for i, enum in enumerate(k_fold_histories):
    plt.plot(enum.history['loss'])
    plt.title("Fold " + str(i) + " Loss")
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.show()
    plt.savefig("Model_Data/Fold_" + str(i) + "_Loss.png")


KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
#Print classification summaries for each fold
clr_file = open("Model_Data/Fold_Test_Classification_Reports.txt", 'w')
for i, enum in enumerate(k_fold_classification_reports):
    print("Fold " + str(i) + " Classification Report")
    print("-------------------------------------------")
    print(enum + "\n")
    clr_file.write("Fold " + str(i) + " Classification Report"+"\n")
    clr_file.write("-------------------------------------------"+"\n")
    clr_file.write(enum + "\n")
clr_file.close()

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
model.save('arthro_multiclass_model.h5')

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
model = tf.keras.models.load_model('arthro_multiclass_model.h5')
print(model.summary())

KernelInterrupted: Execution interrupted by the Jupyter kernel.

KernelInterrupted: Execution interrupted by the Jupyter kernel.